In [1]:
import pyodbc 
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("./PlayerSeason.2024.csv")

print(df.head())
print(df.shape)

# TODO: https://www.dropbox.com/scl/fi/hy0sxw6gaai7ghemrshi8/lahman_1871-2023_csv.7z?dl=0&e=1&file_subpath=%2Flahman_1871-2023_csv&rlkey=edw1u63zzxg48gvpcmr3qpnhz

    StatID  TeamID  PlayerID  SeasonType  Season               Name Team  \
0  6422574       1  10008679           2    2024  Elieser Hernandez  LAD   
1  6422575       1  10013631           2    2024         Drew Avans  LAD   
2  6422576       1  10009292           2    2024      Michael Grove  LAD   
3  6422577      33  10007188           2    2024      Tirso Ornelas   SD   
4  6422578       1  10010357           2    2024         Kody Hoese  LAD   

  Position PositionCategory  Started  ...  PitchingCompleteGames  \
0       SP                P        0  ...                    0.0   
1       CF               OF        2  ...                    0.0   
2       SP                P        1  ...                    0.0   
3       LF               OF        0  ...                    0.0   
4       3B               IF        0  ...                    0.0   

   PitchingShutOuts  PitchingOnBasePercentage  PitchingSluggingPercentage  \
0               0.0                     0.455            

In [ ]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = '' 

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:

df = df.dropna(subset=['PositionCategory'])
df = df.replace(np.nan, None)

print(type(df['BirthDate']))

print(df.shape)

In [ ]:
for index, row in df.iterrows():
		
		cursor.execute("SELECT t.TeamID FROM [MLB].[Teams] t WHERE Abbreviation = ?", row.Team)
		team = cursor.fetchone()

		playerTeam = int(team[0]) if team is not None else None

		date = None

		if (row.BirthDate is not None):
			date = datetime.strptime(row.BirthDate, "%m/%d/%Y %H:%M:%S %p")
																																																																																												#11/30/1987 12:00:00 AM
		cursor.execute("INSERT INTO MLB.Players (FirstName,LastName,Jersey,PositionCategory,Position,Status,TeamID, DateOfBirth) VALUES(?,?,?,?,?,?,?,?)", row.FirstName, row.LastName, row.Jersey, row.PositionCategory, row.Position, row.Status, playerTeam, date)

cnxn.commit()
cursor.close()